In [1]:
from sagemaker.estimator import Estimator
from sagemaker.session import Session
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
role = get_execution_role()
sess = Session()

In [3]:
estimator = Estimator(
    image_uri='405894843732.dkr.ecr.ap-northeast-2.amazonaws.com/automl-sm-papermill-tabular312-langchain:1.0',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    hyperparameters={
        "table_name": "automl-classification-experiment",
        "project_hashkey": "2ee07a49",
        "experiment_hashkey": "1cbd8309",
        "dataset_table_name": "automl-dataset",
        "dataset_profile_table_name": "automl-dataset-profile-experiment-result",
        "model_repo_table_name": "automl-model-repo",
        "model_experiment_result_table_name": "automl-classification-experiment",
        "username": "sean@gs.co.kr",
        "job_type": "training",
        "task_token": "1234",
    },
    base_job_name='custom-training',
    sagemaker_session=sess
)

estimator.fit()

INFO:sagemaker:Creating training-job with name: custom-training-2026-02-26-08-36-32-188
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-training-job


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:22                                                                                   │
│                                                                                                  │
│   19 │   sagemaker_session=sess                                                                  │
│   20 )                                                                                           │
│   21                                                                                             │
│ ❱ 22 estimator.fit()                                                                             │
│   23                                                                                             │
│                                                                                                  │
│ /home/ec2-user/SageMaker/.myenv/miniconda/envs/tabular312/lib/python3.12/site-packages/sagemaker │
│ /workflow/pipeline_context.py:346 in wrapper                                                     │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                                                            │
│                                                                                                  │
│ /home/ec2-user/SageMaker/.myenv/miniconda/envs/tabular312/lib/python3.12/site-packages/sagemaker │
│ /estimator.py:1368 in fit                                                                        │
│                                                                                                  │
│   1365 │   │   self._prepare_for_training(job_name=job_name)                                     │
│   1366 │   │                                                                                     │
│   1367 │   │   experiment_config = check_and_get_run_experiment_config(experiment_config)        │
│ ❱ 1368 │   │   self.latest_training_job = _TrainingJob.start_new(self, inputs, experiment_confi  │
│   1369 │   │   self.jobs.append(self.latest_training_job)                                        │
│   1370 │   │   forward_to_mlflow_tracking_server = False                                         │
│   1371 │   │   if os.environ.get("MLFLOW_TRACKING_URI") and self.enable_network_isolation():     │
│                                                                                                  │
│ /home/ec2-user/SageMaker/.myenv/miniconda/envs/tabular312/lib/python3.12/site-packages/sagemaker │
│ /estimator.py:2506 in start_new                                                                  │
│                                                                                                  │
│   2503 │   │   train_args = cls._get_train_args(estimator, inputs, experiment_config)            │
│   2504 │   │                                                                                     │
│   2505 │   │   logger.debug("Train args after processing defaults: %s", train_args)              │
│ ❱ 2506 │   │   estimator.sagemaker_session.train(**train_args)                                   │
│   2507 │   │                                                                                     │
│   2508 │   │   return cls(estimator.sagemaker_session, esti

In [ ]:
# 이후, 최신 트레이닝 잡 정보 접근
training_job_name = estimator.latest_training_job.name
print("Training Job Name:", training_job_name)

# CloudWatch 로그 그룹과 스트림 이름 가져오기
log_group = f"/aws/sagemaker/TrainingJobs"
log_stream = f"{training_job_name}/algo-1"

print("Log Group:", log_group)
print("Log Stream:", log_stream)

In [ ]:
import boto3

logs_client = boto3.client("logs", region_name="ap-northeast-2")

In [ ]:
streams = logs_client.describe_log_streams(
    logGroupName="/aws/sagemaker/TrainingJobs",
    logStreamNamePrefix=f"{training_job_name}/algo-1"
)

for stream in streams['logStreams']:
    print(stream['logStreamName'])  # 전체 UUID 포함된 스트림 이름
